# Setup Paths

In [16]:
import os

def setup_paths(dir_location, output_to_cloud, run_tag, is_original_data):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')
    
    base_gt_path = os.path.join(base_path, 'database')
    if is_original_data:
        gt_path = os.path.join(base_gt_path, 'orignal_dataset', 'segmented', 'tiff')
    else:
        gt_path = os.path.join(base_gt_path, 'tablet_dataset', 'segmented', 'tiff')
        
    base_output_path = os.path.join(base_path, 'outputs')
    output_path = os.path.join(base_output_path, 'tiff', run_tag)
    if output_to_cloud:
        base_output_path = os.path.join(r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files', 'outputs')
    results_path = os.path.join(base_output_path, 'metrics', 'ratio_detected')
    if not os.path.isdir(results_path):
        os.makedirs(results_path)

    print('Paths set')
    return gt_path, output_path, results_path

# Extract Pixel Values from Ground Truth TIFF Stack

In [17]:
import numpy as np
import tifffile as tiff
import os
import matplotlib.pyplot as plt

def extract_unique_pixel_values(folder_path):
    """Extract unique pixel values and their counts from a TIFF stack in a folder."""
    pixel_values = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            img = tiff.imread(os.path.join(folder_path, filename))
            pixel_values.append(img.flatten())  # Flatten to a 1D array
    
    # Concatenate all pixel values and get unique values with counts
    all_pixels = np.concatenate(pixel_values)
    unique_values, counts = np.unique(all_pixels, return_counts=True)
    
    return unique_values, counts  # Return both unique values and their frequencies

def highlight_pixel_value(folder_path, target_value):
    """Display an image where only a specific pixel value is shown, using the middle file in the folder."""
    tiff_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif') or f.endswith('.tiff')])
    
    if not tiff_files:
        raise FileNotFoundError("No TIFF files found in the folder.")
    
    # Select the middle file
    middle_file = tiff_files[len(tiff_files) // 2]
    img = tiff.imread(os.path.join(folder_path, middle_file))  # Load the middle file
    
    # Create a mask where only the target pixel value is highlighted
    highlighted_img = np.where(img == target_value, target_value, 0).astype(np.uint8)
    
    plt.imshow(highlighted_img, cmap="gray")
    plt.axis("off")
    plt.show()  # Display without saving

def load_tiff_stack_from_dir(directory):
    """Loads a stack of TIFF images from a directory into a 3D NumPy array."""
    tiff_files = sorted([f for f in os.listdir(directory) if f.lower().endswith('.tiff') or f.lower().endswith('.tif')])
    
    if not tiff_files:
        raise FileNotFoundError("No TIFF files found in the directory.")

    stack = [tiff.imread(os.path.join(directory, f)) for f in tiff_files]
    return np.stack(stack, axis=0)  # Stack images along the first dimension (depth)

In [18]:
# tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\5_ClaritinD12'
# pixel_values, counts = extract_unique_pixel_values(tiff_path)
# print(pixel_values)
# print(counts)

In [19]:
# tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\5_ClaritinD12_Aug3'
# pixel_values, counts = extract_unique_pixel_values(tiff_path)
# print(pixel_values)
# print(counts)

In [20]:
# tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\5_ClaritinD12_Aug4'
# pixel_values, counts = extract_unique_pixel_values(tiff_path)
# print(pixel_values)
# print(counts)

In [21]:
# tiff_path = r'd:\Senior_Design\database\tablet_dataset\segmented\tiff\5_ClaritinD12'
# highlight_pixel_value(tiff_path, target_value=159)

# Percent Detected 

In [22]:
import os
import numpy as np
import pandas as pd
import tifffile as tiff
from pathlib import Path
from tqdm import tqdm

def process_image(img, gt_path, output_path, run_tag, to_binary=False):
    gt_stack_path = os.path.join(gt_path, img)
    output_stack_path = os.path.join(output_path, img)
    
    if img == '2_Tablet':
        gt_val = 255
    elif img == '4_GenericD12':
        gt_val = 182
    elif img == '5_ClaritinD12':
        gt_val = 159
    else:
        raise ValueError('No valid ground truth value stored for this tablet type')
    
    gt_files = sorted([os.path.join(gt_stack_path, f) for f in os.listdir(gt_stack_path) if f.lower().endswith('.tiff')])
    model_files = sorted([os.path.join(output_stack_path, f) for f in os.listdir(output_stack_path) if f.lower().endswith('.tiff')])

    total_in_gt = 0
    total_in_model = 0
    
    for gt_path, model_path in tqdm(list(zip(gt_files, model_files)), desc=f"Processing {img}", total=len(gt_files)):
        gt_image = tiff.imread(gt_path)
        model_image = tiff.imread(model_path)

        if to_binary:
            model_image = np.where(model_image > 0, 255, 0)
        else:
            if len(extract_unique_pixel_values(output_stack_path)) > 2:
                raise ValueError('The output TIFF stack of the model is not a binary mask. This will not work without a binary mask (for now at least).')
        
        if gt_image.shape != model_image.shape:
            raise ValueError(f"Shape mismatch between {gt_path} and {model_path}.")
        
        gt_mask = gt_image == gt_val
        total_in_gt += np.sum(gt_mask)
        total_in_model += np.sum((model_image == 255) & gt_mask)
    
    ratio = total_in_model / total_in_gt if total_in_gt > 0 else np.nan
    return [run_tag, img, total_in_gt, total_in_model, ratio]

def ratio_detected(img_names=None, dir_location='internal', output_to_cloud=False, run_tag='No Run Tag Inputted', is_original_data=False, to_binary=True):
    gt_path, output_path, results_path = setup_paths(dir_location, output_to_cloud, run_tag, is_original_data)
    
    if img_names is None:
        img_names = [f.name for f in Path(output_path).iterdir() if f.is_dir()]  # List of directories
    
    results = []
    for img in img_names:
        results.append(process_image(img, gt_path, output_path, run_tag, to_binary))
    
    df = pd.DataFrame(results, columns=['run_tag', 'image_name', 'number_in_gt', 'number_in_model', 'model_to_gt_ratio'])
    
    results_file = os.path.join(results_path, 'ratio_detected.csv')
    df['model_to_gt_ratio'] = df['model_to_gt_ratio'].astype(float)
    if os.path.exists(results_file):
        df.to_csv(results_file, mode='a', header=False, index=False)
    else:
        df.to_csv(results_file, index=False)
    
    return df


In [23]:
# img_names = ['2_Tablet']
# img_names = ['4_GenericD12']
# img_names = ['5_ClaritinD12']
img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']
dir_location = 'refine'
# run_tag = 'pretrained_gen35_zscore4'
# run_tag = 'pretrained_clar35_zscore5'
run_tag = 'pretrained_tab40_gen35_clar35_fold1'
# run_tag = 'pretrained_tab40_gen35_clar35_foldsALL'

ratio_detected(img_names=img_names, dir_location=dir_location, run_tag=run_tag)

Paths set


Processing 2_Tablet:   0%|          | 0/950 [00:00<?, ?it/s]

Processing 5_ClaritinD12: 100%|██████████| 1008/1008 [00:08<00:00, 123.08it/s]


,run_tag,image_name,number_in_gt,number_in_model,model_to_gt_ratio
0,pretrained_tab40_gen35_clar35_fold1,2_Tablet,31224949,20744222,0.664348
1,pretrained_tab40_gen35_clar35_fold1,4_GenericD12,41820600,21642841,0.517516
2,pretrained_tab40_gen35_clar35_fold1,5_ClaritinD12,42293472,24685982,0.583683
